In [1]:
# Declaring path variables to make Basemap work in jupyter hub. Make sure that the conda environment path matches yours
import os
os.environ['PATH'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/bin:' + os.environ['PATH']
os.environ['PROJ_LIB'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/share/proj'

# General Tool Imports
import numpy as np
import pandas as pd
import collections
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
import math
import pickle 

import lasio



%matplotlib inline


In [2]:
tracers = pd.read_excel('Full Chaparral Tracer Data with Formations.xlsx')

In [13]:
formdict = {
    '"Foraker 10UMH"':'Meramec E',
    '"Foraker 11SMH"':'Sycamore',
    '"Foraker 1SMH"':'Sycamore',
    '"Foraker 2UMH"':'Meramec E',
    '"Foraker 3WH"':'Woodford',
    '"Foraker 4LMH"':'Meramec A',
    '"Foraker 5WMH"':'Woodford',
    '"Foraker 6UMH"':'Meramec E',
    '"Foraker 7SMH"':'Sycamore',
    '"Foraker 8UMH"':'Meramec E',
    '"Foraker 9LMH"':'Meramec A'
}

In [6]:
tracers.head()

,TracerVision File Name,Sample ID,Tracerco Sample #,Tracer Type,Well Applied to,Well Produced From,Stage,Stage Applied to,Stage Produced From,Production Date - Copy,...,Cum Oil (bbls),Gas Prod (MMSCFD),Cum Gas (mcf),Water Prod (BWPD),Cum Water (bbls),Choke,Tracer Production,Cumulative Tracer Production,Stage Flow Rate (BBL/Day),Stage Cum Production (BBLS)
0,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2539,140461,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/5/2019 3:00:00 PM""",...,150.0,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN
1,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2540,140462,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/5/2019 11:00:00 PM""",...,300.0,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN
2,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2541,140463,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/6/2019 12:00:00 PM""",...,469.0,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN
3,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2542,140464,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/6/2019 11:00:00 PM""",...,638.0,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN
4,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2543,140465,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/7/2019 12:00:00 PM""",...,878.0,NaN,NaN,NaN,NaN,0,0.0,0.0,NaN,NaN


In [14]:
tracers['Prod From FM'] = tracers['Well Produced From'].map(formdict)
tracers['Applied to FM'] = tracers['Well Applied to'].map(formdict)


In [15]:
tracers.head()

,TracerVision File Name,Sample ID,Tracerco Sample #,Tracer Type,Well Applied to,Well Produced From,Stage,Stage Applied to,Stage Produced From,Production Date - Copy,...,Water Prod (BWPD),Cum Water (bbls),Choke,Tracer Production,Cumulative Tracer Production,Stage Flow Rate (BBL/Day),Stage Cum Production (BBLS),Formation,Prod From FM,Applied to FM
0,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2539,140461,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/5/2019 3:00:00 PM""",...,NaN,NaN,0,0.0,0.0,NaN,NaN,Sycamore,Meramec A,Sycamore
1,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2540,140462,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/5/2019 11:00:00 PM""",...,NaN,NaN,0,0.0,0.0,NaN,NaN,Sycamore,Meramec A,Sycamore
2,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2541,140463,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/6/2019 12:00:00 PM""",...,NaN,NaN,0,0.0,0.0,NaN,NaN,Sycamore,Meramec A,Sycamore
3,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2542,140464,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/6/2019 11:00:00 PM""",...,NaN,NaN,0,0.0,0.0,NaN,NaN,Sycamore,Meramec A,Sycamore
4,TV Workbook - Foraker Pad 3 - Final V3.xlsm,2543,140465,Oil,"""Foraker 11SMH""","""Foraker 4LMH""",1,"""Foraker 11SMH_1""","""Foraker 4LMH_1""","""4/7/2019 12:00:00 PM""",...,NaN,NaN,0,0.0,0.0,NaN,NaN,Sycamore,Meramec A,Sycamore


In [18]:
tracers.to_csv('TracerDatawithFormations.csv')